In [1]:
from datetime import datetime
import pandas as pd
from trainingData import trainingData

predictionSymbol = '^GSPC' #'^GSPC' #'^IXIC' #symbol for S&P500 (^GSPC) or NASDAQ (^IXIC) or AUS ('^AXJO')
startdate = datetime(2013,10,1)
enddate = datetime(2016,10,11)
teststartdate = datetime(2016,7,1)
delta = range(2, 5)
lags = range(2, 3)

td = trainingData()
td.load(predictionSymbol, startdate, enddate, delta, lags)

got nasdaq
got sp500
got volatility
got australia
got hkong


C:\git\github\stockmarket-predictor\stockmarket-predictor\featureGenerator.py:18: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=2,center=False).mean()
  dataframe[roll_n] = pd.rolling_mean(dataframe[returns], n) #pd.Series(dataframe[returns]).rolling(window=n).mean()
C:\git\github\stockmarket-predictor\stockmarket-predictor\featureGenerator.py:18: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
  dataframe[roll_n] = pd.rolling_mean(dataframe[returns], n) #pd.Series(dataframe[returns]).rolling(window=n).mean()
C:\git\github\stockmarket-predictor\stockmarket-predictor\featureGenerator.py:18: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=4,center=False).mean()
  dataframe[roll_n] = pd.rolling_mean(dataframe[r

In [2]:
#ds = td.loadAllFromWeb(predictionSymbol, startdate, enddate)

In [3]:
#for column in ds[0].columns:
#    ds[0][column] = ds[0][column].shift(1)
#ds[0]

In [4]:
#import featureGenerator as fg
#wfeat = fg.applyRollMeanDelayedReturns(ds, delta)

In [5]:
testsize = 0.1
X_train, y_train, X_test, y_test = td.returnDataForClassification(teststartdate, testsize)

In [6]:
y_test.size

69

In [7]:
# score accuracy of each algorithm
import algorithms as alg
accuracy, rfc = alg.performRFClass(X_train, y_train, X_test, y_test)
print('RFC =', accuracy)
accuracy, knn = alg.performKNNClass(X_train, y_train, X_test, y_test)
print('KNN =', accuracy)
accuracy, svm = alg.performSVMClass(X_train, y_train, X_test, y_test)
print('SVM =', accuracy)
accuracy, ada = alg.performAdaBoostClass(X_train, y_train, X_test, y_test)
print('ADA =', accuracy)
accuracy, gtb = alg.performGTBClass(X_train, y_train, X_test, y_test)
print('GTB =', accuracy)
#accuracy = alg.performQDAClass(X_train, y_train, X_test, y_test)
#print('QDA =', accuracy)
accuracy, mlp = alg.performMLPClass(X_train, y_train, X_test, y_test, (5,6,2))
print('MLP =', accuracy)

RFC = 0.579710144928
KNN = 0.565217391304
SVM = 0.579710144928
ADA = 0.536231884058
GTB = 0.507246376812
MLP = 0.608695652174


# Evaluate for single prediction

In [9]:
# evaluate for single predictions across all algorithms
startdate = datetime(2016,10,1)
enddate = datetime(2016,10,12)
delta = range(2, 5)
lags = range(2, 3)

td = trainingData()
td.load(predictionSymbol, startdate, enddate, delta, lags)
predData = td.returnDataForPrediction()

got nasdaq
got sp500
got volatility
got australia
got hkong


In [31]:
predData = predData[:1]
predData

,OutTime2,OutRolMean2,OutTime3,OutRolMean3,OutTime4,OutRolMean4,Return_^AXJO,^AXJOTime2,^AXJORolMean2,^AXJOTime3,...,^HSITime3,^HSIRolMean3,^HSITime4,^HSIRolMean4,Return_Out2,Return_^AXJO2,Return_^IXIC2,Return_^GSPC2,Return_^VIX2,Return_^HSI2
Date,,,,,,,,,,,,,,,,,,,,,
2016-10-07,-0.002773,-0.001386,0.001511,0.000508,-0.003452,-0.000858,0.001463,-0.001386,-0.000691,0.004126,...,-0.010038,-0.003325,-0.005906,-0.00145,0.000482,0.00552,0.004983,0.004297,-0.046955,0.006902


In [30]:

#td.finalTrainingData.columns[1:].size
prediction = rfc.predict(predData)
print('rfc prediction =', prediction)
prediction = knn.predict(predData)
print('knn prediction =', prediction)
prediction = svm.predict(predData)
print('svm prediction =', prediction)
prediction = ada.predict(predData)
print('ada prediction =', prediction)
prediction = gtb.predict(predData)
print('gtb prediction =', prediction)
prediction = mlp.predict(predData)
print('mlp prediction =', prediction)

rfc prediction = [1]
knn prediction = [0]
svm prediction = [1]
ada prediction = [1]
gtb prediction = [1]
mlp prediction = [1]


# Back test

In [49]:
from datetime import datetime
import pandas as pd
from trainingData import trainingData

predictionSymbol = '^GSPC' #'^GSPC' #'^IXIC' #symbol for S&P500 (^GSPC) or NASDAQ (^IXIC) or AUS ('^AXJO')

startdate = datetime(2016,1,4)
enddate = datetime(2016,10,13)
delta = range(2, 5)
lags = range(2, 3)

td = trainingData()
td.load(predictionSymbol, startdate, enddate, delta, lags)
predData = td.returnDataForPrediction()

# symbol of the stock required for future plotting
symbol = 'S&P-500'

# name of the file of the output of prediction (S&P 500 in this case)
name = 'sp500-2016.csv'

# calls the best model previously saved in pickle file and runs it on the test set retutning an array of 0,1 (Down, Up) according to predicted returns
#prediction = pystocks.getPredictionFromBestModel(9, 9, 'sp500', cut, start_test, path_datasets, 'sp500_57.pickle')[0]
prediction = mlp.predict(predData)

# dataframe of S&P 500 historical prices (saved locally from Yahho Finance)
bars = pd.read_csv(name, index_col=0, parse_dates=True)    

# subset of the data corresponding to test set
bars = pd.merge(bars, predData, how='inner', on=None, left_index=True, right_index=True).columns[0:6]

# initialize empty dataframe indexed as the bars. There's going to be perfect match between dates in bars and signals 
signals = pd.DataFrame(index=bars.index)

# initialize signals.signal column to zero
signals['signal'] = 0.0

# copying into signals.signal column results of prediction
signals['signal'] = prediction

# replace the zeros with -1 (new encoding for Down day)
signals.signal[signals.signal == 0] = -1

# compute the difference between consecutive entries in signals.signal. As
# signals.signal was an array of 1 and -1 return signals.positions will 
# be an array of 0s and 2s.
signals['positions'] = signals['signal'].diff()     
bars

got nasdaq
got sp500
got volatility
got australia
got hkong


C:\git\github\stockmarket-predictor\stockmarket-predictor\featureGenerator.py:18: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=2,center=False).mean()
  dataframe[roll_n] = pd.rolling_mean(dataframe[returns], n) #pd.Series(dataframe[returns]).rolling(window=n).mean()
C:\git\github\stockmarket-predictor\stockmarket-predictor\featureGenerator.py:18: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
  dataframe[roll_n] = pd.rolling_mean(dataframe[returns], n) #pd.Series(dataframe[returns]).rolling(window=n).mean()
C:\git\github\stockmarket-predictor\stockmarket-predictor\featureGenerator.py:18: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=4,center=False).mean()
  dataframe[roll_n] = pd.rolling_mean(dataframe[r

,Open_Out,High,Low,Close_Out,Volume,Adj Close,OutTime2,OutRolMean2,OutTime3,OutRolMean3,...,^HSITime3,^HSIRolMean3,^HSITime4,^HSIRolMean4,Return_Out2,Return_^AXJO2,Return_^IXIC2,Return_^GSPC2,Return_^VIX2,Return_^HSI2
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-08,1945.969971,1960.400024,1918.459961,1922.030029,4664940000,1922.030029,-0.034282,-0.017269,-0.046948,-0.015885,...,-0.052062,-0.017525,-0.061364,-0.015597,-0.023700,-0.022018,-0.011381,-0.013115,0.064633,-0.030860
2016-01-11,1926.119995,1935.650024,1901.099976,1923.670044,4607290000,1923.670044,-0.009994,-0.004993,-0.033458,-0.011229,...,-0.030570,-0.010200,-0.060486,-0.015365,-0.010838,-0.003892,-0.030260,-0.023700,0.213696,0.005920
2016-01-12,1927.829956,1947.380005,1914.349976,1938.680054,4887260000,1938.680054,0.008663,0.004328,-0.002270,-0.000727,...,-0.025366,-0.008400,-0.019596,-0.004820,0.000853,-0.011742,-0.009767,-0.010838,0.080832,-0.027634
2016-01-13,1940.339966,1950.329956,1886.410034,1890.280029,5087030000,1890.280029,-0.017357,-0.008581,-0.016519,-0.005436,...,-0.003574,-0.001153,-0.031109,-0.007773,0.007803,-0.001440,-0.001214,0.000853,-0.100333,-0.008887
2016-01-14,1891.680054,1934.469971,1878.930054,1921.839966,5241110000,1921.839966,-0.008686,-0.004135,-0.000951,-0.000156,...,-0.009689,-0.003181,-0.018490,-0.004607,-0.024965,0.012649,0.010334,0.007803,-0.075309,0.011319
2016-01-15,1916.680054,1916.680054,1857.829956,1880.329956,5468460000,1880.329956,-0.005264,-0.002452,-0.030098,-0.009956,...,-0.034985,-0.011792,-0.024062,-0.006014,0.016696,-0.015639,-0.034115,-0.024965,0.122385,-0.005893
2016-01-19,1888.660034,1901.439941,1864.599976,1881.329956,4928350000,1881.329956,-0.021079,-0.010534,-0.004735,-0.001457,...,-0.032502,-0.010659,-0.046984,-0.011736,-0.021599,-0.003381,0.019651,0.016696,-0.050357,-0.014969
2016-01-20,1876.180054,1876.180054,1812.290039,1859.329956,6416070000,1859.329956,-0.011168,-0.005581,-0.032526,-0.010920,...,-0.036143,-0.011895,-0.050132,-0.012550,0.000532,-0.006969,-0.027428,-0.021599,0.128184,-0.014514
2016-01-21,1861.459961,1889.849976,1848.979980,1868.989990,5078810000,1868.989990,-0.006559,-0.003249,-0.006031,-0.001989,...,-0.028280,-0.009119,-0.008158,-0.001663,-0.011694,-0.012563,-0.002555,0.000532,-0.035899,-0.038171


In [52]:
#print(len(bars))
#print(len(predData))
#print(len(bars))
##predData
#bars= bars.columns[0:-8]
#bars

AttributeError: 'Index' object has no attribute 'columns'

In [53]:
from portfolio import MarketIntradayPortfolio

# calling portfolio evaluation on signals (predicted returns) and bars 
# (actual returns)
portfolio = MarketIntradayPortfolio(symbol, bars, signals)

# backtesting the portfolio and generating returns on top of that 
returns = portfolio.backtest_portfolio()
returns

C:\Program Files\Anaconda3\envs\smp\lib\site-packages\pandas\indexes\base.py:1264: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return getitem(key)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices